In [1]:
import pandas as pd
import numpy as np
import sumu
import pandas as pd
import numpy as np
from glmnet import ElasticNet
import random
import time
import random

/home/blazej/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/home/blazej/.local/lib/python3.6/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [10]:
def generate_dataset_and_dag(dim,number, deg):
    dictionary = {}
    for x in range(1,dim+1):
        dictionary[x] = np.random.normal(0,1,number)
    df = pd.DataFrame(dictionary)
    possible_children =set(range(1,dim+1))
    possible_parents =set()
    root = random.sample(possible_children, k=1)[0]
    possible_children.remove(root)
    possible_parents.add(root)
    dag = dict()
    for x in range(1,dim+1):
        dag[x]=[]
    for x in range(1,dim):
        if len(possible_children) !=0:
            new_children = list(set(random.choices(list(possible_children), k=deg)))
        else:
            break
        their_parent = random.sample(possible_parents, k=1)[0]
        relations = list()
        for child in new_children:
            possible_parents.add(child)
            possible_children.remove(child)
            rand_weight = random.random()
            df[child] = df[their_parent]*rand_weight
            relations.append((child,rand_weight))
        dag[their_parent]+=relations
    
    return np.array(df),dag
        
        
arr1, dag1 = generate_dataset_and_dag(5,1000000,2)        

In [11]:
print(dag1)
avg = np.average(arr1,axis=0)
print(avg)

{1: [(2, 0.4590949461180833)], 2: [], 3: [], 4: [], 5: [(1, 0.05467791121160703), (3, 0.7679515539259062), (4, 0.9280766664604173)]}
[1.36003292e-05 6.24384239e-06 1.91016696e-04 2.30845471e-04
 2.48735346e-04]


In [12]:
start = time.time()
data = sumu.Data(arr1, discrete=False)
params = {

          "array": arr1,
          "data": data,
          "scoref": "bge",  # Or "bdeu" for discrete data.
          # "ess": 10,        # If using BDeu.
          "max_id": -1,
          "K": 2,
          "d": 2,
          "cp_algo": "greedy-lite",
          "mc3_chains": 10,
          "burn_in": 100,
          "iterations": 10000,
          "thinning": 10}

g = sumu.Gadget(**params)
dag = g.sample()
end = time.time()
print(dag)
print(dag1)
print(end - start)


/home/blazej/sumu/sumu/mcmc.py:614: RuntimeWarning: overflow encountered in exp
  if np.random.rand() < np.exp(self.temp * sum(R_prime_node_scores) - self.R_score)*q_rev/q:


{3: [(4, 0.9010227391693756)], 0: [(3, 0.05719788354163362), (4, 2.3373481821023856e-13)], 1: [(3, 0.02625925926261313), (4, 1.0876054980084648e-13)], 2: [(3, 0.8033445787106783), (4, 3.3211816460064055e-12)]}
{1: [(2, 0.4590949461180833)], 2: [], 3: [], 4: [], 5: [(1, 0.05467791121160703), (3, 0.7679515539259062), (4, 0.9280766664604173)]}
27.52322030067444


In [ ]:
for x in dag:
    if x in dag1:
        print('______',x)
        print(dag1[x])
        print(dag[x])
    